<a href="https://colab.research.google.com/github/Hurabashi/Matricu_Daugyba_su_Strassen/blob/main/Matricu_daugyba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Šiame projekte lyginsime paprastą matricų sandaugos algoritmą ir matricų sandaugos Strassen algoritmą.

Pirma pradėsime nuo paprastos matricų sandaugos.

In [ ]:
def matricu_sandauga_paprasta(matrica_1,matrica_2):
  '''
  Iprastu ir neoptimizuotu budu sudauginame dvi matricas, veikia jeigu sutampa
  pirmosios plotis ir antrosios aukstis/ilgis.
  '''
  suma = 0
  galutine_matrica = []
  for eilute in matrica_1:
    ilgis_eil = len(eilute)
    ilgis_stulp = len(matrica_2[0])
    galutines_matricos_eilute = []
    for i in range(ilgis_stulp):
      for j in range(ilgis_eil):
        suma = suma + eilute[j] * matrica_2[j][i]
      galutines_matricos_eilute.append(suma)
      suma = 0 
    galutine_matrica.append(galutines_matricos_eilute)
  return galutine_matrica
  
  

Paruosiame matricas tinkamas Strassen algoritmui, jeigu jos nebūtų kvadratinės ir/arba 2^n. Mes pridedame nulių, kad jas tokiomis paversti.
(Pastaba: užduoty naudojamos tik kvadratinės matricos, čia veiks su visomis daugintinomis matricomis)

In [ ]:
def matricu_kvadratinimas(matrica_1, matrica_2):
  '''
  Is abieju matricu randame didziausia dimensija, tada randame tai dimensijai
  maziausia 2^n, ir tada abieju matricu dimensijas ispleciame iki jos, pridedant
  nulius.
  '''
  #ieskome kokia didziausia dimensija
  didziausia_dimensija = max(len(matrica_1[0]),len(matrica_1),
                    len(matrica_2[0]),len(matrica_2))
  #padarome didziausia dimensija i 2^n 
  while (didziausia_dimensija & (didziausia_dimensija - 1)) != 0:
         didziausia_dimensija = didziausia_dimensija + 1

  #paplatiname matricas iki didziausios dimensijos
  #prailginame eilutes nuliais
 
  for i in range(didziausia_dimensija - len(matrica_1[0])):
    matrica_1[0].append(0)

  for i in range(didziausia_dimensija - len(matrica_2[0])):
    matrica_2[0].append(0)
  #sukuriam nuliu eilutes
  nuliai = []
  for i in range(didziausia_dimensija):
    nuliai.append(0)
  
  #prideda nuliu eilutes
  while len(matrica_1) != didziausia_dimensija:
    matrica_1.append(nuliai)
  while len(matrica_2) != didziausia_dimensija:
    matrica_2.append(nuliai)


 

Toliau mes apsibrėžiame pagalbines funkcijas: matricų sudėtį, atimtį, išardymą į 4 mažesnias matricas, bei sujungimo atgal į vieną. Aišku tai reikalinga, nes mes nesinaudojame išorinėmis bibliotekomis kaip numpy, kurios jau turi apibrėžtą dalį operacijų bei palengvina kitas, bet su jomis algoritmų sudėtingumo lyginimas taptų gan beprasmis, nes jos parašytos kitomis programavimo kalbomis ir tai nebūtų savarankiškas darbas.

In [ ]:
def sudetis(x, y):
  #tikrinama ar musu atveju tai matrica jei taip atlieka matricu sudetis
  if isinstance(x, list):
    c = []
    plotis = len(x[0])
    ilgis = len(x)
    for i in range(ilgis):
      c_eil = []
      for j in range(0, plotis): 
        c_eil.append(x[i][j] + y[i][j])
      c.append(c_eil)
    return c
  else:
    return x + y

def atimtis(x, y):
  #tikrinama ar musu atveju tai matrica jei taip tai atliekama matricu atimtis
  if isinstance(x, list):
    c = []
    plotis = len(x[0])
    ilgis = len(x)
    for i in range(ilgis):
      c_eil = []
      for j in range(0, plotis): 
        c_eil.append(x[i][j] - y[i][j])
      c.append(c_eil)
    return c
  else:
    return x - y

def matricos_padalinimas(matrica):
  #padalinime matrica i 4 matricas
  dimensija = len(matrica)
  dimensija_galutine = int(dimensija/2)

  mat_00 = []
  for eil in matrica[0 : dimensija_galutine]:
    mat_00.append(eil[0 : dimensija_galutine])
    
  mat_01 = []
  for eil in matrica[0 : dimensija_galutine]:
    mat_01.append(eil[dimensija_galutine : dimensija])
    
  mat_10 = []
  for eil in matrica[dimensija_galutine : dimensija]:
    mat_10.append(eil[0 : dimensija_galutine])
    
  mat_11 = []
  for eil in matrica[dimensija_galutine : dimensija]:
    mat_11.append(eil[dimensija_galutine : dimensija])
  return mat_00, mat_01, mat_10, mat_11
  
def matricos_suvienijimas(mat_00, mat_01, mat_10, mat_11):
  #sujungiame 4 matricas i viena
  if isinstance(mat_00, list):
    c = []
    dimensija = len(mat_00)
    for i in range(dimensija):
      c.append(mat_00[i] + mat_01[i])   
    for i in range(dimensija):
      c.append(mat_10[i] + mat_11[i])
  else:
    c = [[mat_00, mat_01],
        [mat_10, mat_11]]
    
  return c

Pagaliau galime apsibrėžti Strassen rekursinį matricų daugybos algoritmą.

In [ ]:
def matricu_sandauga_strassen(x, y):
  '''
  Dauginame matricas Strassen rekurisniu algoritmu
  '''

  #sukuriame isejimo salyga pasiekus skaliarus
  if len(x) == 1:
    return x[0][0] * y[0][0]

  #padaliname matricas i savus kvadratus
  x_00, x_01, x_10, x_11 = matricos_padalinimas(x)
  y_00, y_01, y_10, y_11 = matricos_padalinimas(y)

  #sukuriame konstantas reikalingas Strassen algoritmui
  m_1 = matricu_sandauga_strassen(atimtis(x_01, x_11), sudetis(y_10, y_11))
  m_2 = matricu_sandauga_strassen(sudetis(x_00, x_11), sudetis(y_00, y_11))
  m_3 = matricu_sandauga_strassen(atimtis(x_00, x_10), sudetis(y_00, y_01))
  m_4 = matricu_sandauga_strassen(sudetis(x_00, x_01), y_11)
  m_5 = matricu_sandauga_strassen(x_00, atimtis(y_01, y_11))
  m_6 = matricu_sandauga_strassen(x_11, atimtis(y_10, y_00))
  m_7 = matricu_sandauga_strassen(sudetis(x_10, x_11), y_00)

  c_00 = sudetis(sudetis(m_1, m_2), atimtis(m_6, m_4))
  c_01 = sudetis(m_4, m_5)
  c_10 = sudetis(m_6, m_7)
  c_11 = sudetis(atimtis(m_2, m_3), atimtis(m_5, m_7))

  #sujungiame i didesne matrica
  c = matricos_suvienijimas(c_00, c_01, c_10, c_11)
  return c

Dabar sukurkime galimybę nuskaityti matricas iš failų ir atlikti daugybą. 

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
failas = 'matricos.txt'
with open(failas, "r") as failas:
    FileasList = failas.readlines()

matrica_did = []

for i in range(len(FileasList)):
  eilute = list(map(int, FileasList[i].split()))
  matrica_did.append(eilute)
  if eilute == []:
    kitos_matricos_zyma = i
  

matrica_1 = matrica_did[0:kitos_matricos_zyma]
matrica_2 = matrica_did[kitos_matricos_zyma+1:]

print('Pasirinkite kokia daugyba norite atlikti Strassen ar paprasta su nuskaitytomis matricomis is failo matricos.txt')
print('Jeigu Strassen, rasykite s simboli, jeigu paprasta - p simboli arba bet ka kita:')
simbolis = input()

if(simbolis == 's'):
  matricu_kvadratinimas(matrica_1,matrica_2)
  galutine_matrica = matricu_sandauga_strassen(matrica_1, matrica_2)
else:
  galutine_matrica = matricu_sandauga_paprasta(matrica_1, matrica_2)

print('Atsakymas:')
for eilute in galutine_matrica:
  print(eilute)

Pasirinkite kokia daugyba norite atlikti Strassen ar paprasta su nuskaitytomis matricomis is failo matricos.txt
Jeigu Strassen, rasykite s simboli, jeigu paprasta - p simboli arba bet ka kita:
aha
Atsakymas:
[196, 162, 150, 157]
[167, 136, 126, 133]
[80, 66, 58, 59]
[179, 151, 156, 154]


Toliau vykdome analizę

In [ ]:
def matricos_generavimas(skaicius, n):
  c = []
  eilute = []
  for sk in range(n):
    eilute.append(skaicius)
  for sk in range(n):
    c.append(eilute)
  
  return c


In [66]:
import time
def testavimas(skaicius, n):

  matrica_1 = matricos_generavimas(skaicius, n)
  matrica_2 = matricos_generavimas(skaicius, n)
  print('sugeneruota')
  matricu_kvadratinimas(matrica_1,matrica_2)
  print('sukvadratinta')
  tic = time.perf_counter()
  matricu_sandauga_strassen(matrica_1, matrica_2)
  toc = time.perf_counter()
  laikas_strassen = toc - tic
  print('strassen')
  tic = time.perf_counter()
  matricu_sandauga_paprasta(matrica_1, matrica_2)
  toc = time.perf_counter()
  laikas_paprasta = toc - tic
  print('paprasta')
  return laikas_paprasta, laikas_strassen


In [74]:
laiku_sarasas = []
for n in range(2, 103, 2):
  print(n)
  pap, stras = testavimas(3, n)
  laiku_sarasas.append([n, pap, stras])

2
sugeneruota
sukvadratinta
strassen
paprasta
4
sugeneruota
sukvadratinta
strassen
paprasta
6
sugeneruota
sukvadratinta
strassen
paprasta
8
sugeneruota
sukvadratinta
strassen
paprasta
10
sugeneruota
sukvadratinta
strassen
paprasta
12
sugeneruota
sukvadratinta
strassen
paprasta
14
sugeneruota
sukvadratinta
strassen
paprasta
16
sugeneruota
sukvadratinta
strassen
paprasta
18
sugeneruota
sukvadratinta
strassen
paprasta
20
sugeneruota
sukvadratinta
strassen
paprasta
22
sugeneruota
sukvadratinta
strassen
paprasta
24
sugeneruota
sukvadratinta
strassen
paprasta
26
sugeneruota
sukvadratinta
strassen
paprasta
28
sugeneruota
sukvadratinta
strassen
paprasta
30
sugeneruota
sukvadratinta
strassen
paprasta
32
sugeneruota
sukvadratinta
strassen
paprasta
34
sugeneruota
sukvadratinta
strassen
paprasta
36
sugeneruota
sukvadratinta
strassen
paprasta
38
sugeneruota
sukvadratinta
strassen
paprasta
40
sugeneruota
sukvadratinta
strassen
paprasta
42
sugeneruota
sukvadratinta
strassen
paprasta
44
sugeneruota
su

In [75]:
laiku_sarasas

[[2, 8.596995030529797e-06, 0.0002112139991368167],
 [4, 2.133099769707769e-05, 0.00030579000303987414],
 [6, 0.00011588900088099763, 0.0029941690008854493],
 [8, 0.00011419100337661803, 0.002085418993374333],
 [10, 0.0004395420037326403, 0.0138440430018818],
 [12, 0.00043308400199748576, 0.009276440003304742],
 [14, 0.00042260299960616976, 0.009156015003100038],
 [16, 0.00043812300282297656, 0.009727234995807521],
 [18, 0.002941702005045954, 0.06500568699993892],
 [20, 0.0029241110023576766, 0.06412393199570943],
 [22, 0.003006304999871645, 0.06542392299888888],
 [24, 0.0029310880054254085, 0.06676999699993758],
 [26, 0.0029395140009000897, 0.0643270080036018],
 [28, 0.0029927499999757856, 0.06486895800480852],
 [30, 0.0030678629991598427, 0.06864517400390469],
 [32, 0.002995140996063128, 0.06951039400155423],
 [34, 0.022385830998246092, 0.45735262500238605],
 [36, 0.02278212399687618, 0.43432459799805656],
 [38, 0.023282362999452744, 0.4477204009963316],
 [40, 0.023970782000105828, 0

Galime matyti, kad bent iš pradžių, žymiai greitesnis yra paprastas algoritmas, bet paziurekime kaip tai vyksta su didesnemis matricomis

In [78]:

laiku_sarasas_did = []
for n in range(2, 500, 25):
  print(n)
  pap, stras = testavimas(3, n)
  laiku_sarasas_did.append([n, pap, stras])

2
sugeneruota
sukvadratinta
strassen
paprasta
27
sugeneruota
sukvadratinta
strassen
paprasta
52
sugeneruota
sukvadratinta
strassen
paprasta
77
sugeneruota
sukvadratinta
strassen
paprasta
102
sugeneruota
sukvadratinta
strassen
paprasta
127
sugeneruota
sukvadratinta
strassen
paprasta
152
sugeneruota
sukvadratinta
strassen
paprasta
177
sugeneruota
sukvadratinta
strassen
paprasta
202
sugeneruota
sukvadratinta
strassen
paprasta
227
sugeneruota
sukvadratinta
strassen
paprasta
252
sugeneruota
sukvadratinta
strassen
paprasta
277
sugeneruota
sukvadratinta
strassen
paprasta
302
sugeneruota
sukvadratinta
strassen
paprasta
327
sugeneruota
sukvadratinta
strassen
paprasta
352
sugeneruota
sukvadratinta
strassen
paprasta
377
sugeneruota
sukvadratinta
strassen
paprasta
402
sugeneruota
sukvadratinta
strassen
paprasta
427
sugeneruota
sukvadratinta
strassen
paprasta
452
sugeneruota
sukvadratinta
strassen
paprasta
477
sugeneruota
sukvadratinta
strassen
paprasta


In [79]:
laiku_sarasas_did

[[2, 6.998001481406391e-06, 3.5178003599867225e-05],
 [27, 0.003272879999713041, 0.08742930099833757],
 [52, 0.02444242999627022, 0.4357755509990966],
 [77, 0.17806212900177343, 3.019750635001401],
 [102, 0.1834354090024135, 3.0617182640053215],
 [127, 0.19392748500104062, 3.00719651899999],
 [152, 1.4222000429945183, 21.124551068001892],
 [177, 1.4438155480020214, 21.299913800998183],
 [202, 1.4769002850007382, 21.859591241998714],
 [227, 1.5218053650023649, 21.45249916699686],
 [252, 1.5544464250051533, 21.28857662399969],
 [277, 11.990764055997715, 149.21179323199613],
 [302, 12.127963618004287, 150.70498701500037],
 [327, 12.209127866000927, 152.601053047998],
 [352, 12.330319950000558, 148.97569100200053],
 [377, 12.392276912003581, 149.31554357300047],
 [402, 12.586353275000874, 149.76429722499597],
 [427, 12.719585136997921, 152.19832926800154],
 [452, 12.894256384002801, 149.20552429399686],
 [477, 12.920140259004256, 149.81096600899764]]